paper: https://link.springer.com/article/10.1007/s11111-024-00452-9#Sec10

Prophet: https://facebook.github.io/prophet/docs/quick_start.html#python-api  
github: https://github.com/facebook/prophet  

| Feature                            | **Prophet**                                    | **STL**                             |
| ---------------------------------- | ---------------------------------------------- | ----------------------------------- |
| **Purpose**                        | Forecasting + decomposition                    | Decomposition only (no forecasting) |
| **Components**                     | Trend + Seasonality + Holidays                 | Trend + Seasonality + Residuals     |
| **Handles missing data?**          | Yes                                            | No (must handle manually)           |
| **Handles irregular time series?** | Yes                                            | No (requires evenly spaced data)    |
| **Forecasting**                    | Yes (core feature)                             | ❌ No forecasting                    |
| **Custom seasonality/holidays**    | Yes (add holidays, custom cycles)              | No                                  |
| **Trend modeling**                 | Piecewise linear or logistic with changepoints | LOESS smoothing (local regression)  |
| **Built-in in Python via**         | `prophet` package                              | `statsmodels.tsa.seasonal.STL`      |


In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# Geospatial
import geopandas as gpd
import dask_geopandas as dgpd
import rasterio
from rasterstats import zonal_stats
from osgeo import gdal, osr

# Statistics & Bayesian modeling
import pymc as pm
import arviz as az

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Base directories
BASE_DIR = Path("/Users/wenlanzhang/PycharmProjects/Mapineq/src/")
DATA_DIR = Path("/Users/wenlanzhang/Downloads/PhD_UCL/Data/Oxford")

In [ ]:
df = pd.read_csv(DATA_DIR/f"Migration/international_migration_flow.csv") 
df['year'] = pd.to_datetime(df['migration_month']).dt.year
df['month'] = pd.to_datetime(df['migration_month']).dt.month
df["migration_month"] = pd.to_datetime(df["migration_month"])

# Define the mapping of old codes to new codes
country_code_mapping = {
    'GR': 'EL',  # Greece (GR → EL)
    'GB': 'UK'   # United Kingdom (GB → UK)
}

# Apply the replacement to both columns
df['country_from'] = df['country_from'].replace(country_code_mapping)
df['country_to'] = df['country_to'].replace(country_code_mapping)

df
# len(df['country_from'].unique())

In [ ]:
# country_list = ['EL', 'ES', 'FI', 'FR', 'HR', 'EE', 'DE', 'DK', 'BA', 'AT', 'BG',
#        'CH', 'CY', 'BE', 'CZ', 'AL', 'LU', 'LV', 'ME', 'IE', 'IS', 'IT',
#        'MK', 'MT', 'LI', 'NL', 'LT', 'HU', 'RS', 'SE', 'SI', 'TR', 'UA',
#        'SK', 'RO', 'NO', 'PL', 'PT', 'XK', 'UK']
# # df = df[(df['country_from'].isin(country_list)) | (df['country_to'].isin(country_list))]
# # df = df[(df['country_from'].isin(country_list)) & (df['country_to'].isin(country_list))]

# # df = df[(df['country_from'] == 'UA') & (df['country_to'].isin(country_list))]
# # df = df[df['num_migrants'] != 0]
# df = df[df['num_migrants'] >= 100]

# df

# Plot

## UA 

In [ ]:
# 1. Filter only rows where destination is UA
df_ua_in = df[df['country_to'] == 'UA']
df_ua_out = df[df['country_from'] == 'UA']

In [ ]:
# # Plot one line per origin country
# plt.figure(figsize=(8, 5))
# for country in df_ua_in['country_from'].unique():
#     subset = df_ua_in[df_ua_in['country_from'] == country]
#     plt.plot(subset['migration_month'], subset['num_migrants'], label=country)
#     # plt.plot(subset['migration_month'], subset['num_migrants'], marker='o', label=country)

# # 3. Add labels and style
# plt.title("Migrants to UA Over Time")
# plt.xlabel("Month")
# plt.ylabel("Number of Migrants")
# # plt.legend(title="From Country")
# plt.grid(True)
# plt.tight_layout()

# plt.show()

In [ ]:
# df_ua_in
fig = px.line(
    df_ua_in,
    x="migration_month",
    y="num_migrants",
    color="country_from",  # separate line for each origin country
    markers=True,
    title="Migrants to UA Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_ua_out
fig = px.line(
    df_ua_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from UA Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### From PL  

In [ ]:
# df_pl_out
df_pl_out = df[(df['country_from'] == 'PL') & (df['num_migrants'] != 0)]

fig = px.line(
    df_pl_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from PL Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### From DE  

In [ ]:
# df_de_out
df_de_out = df[(df['country_from'] == 'DE') & (df['num_migrants'] != 0)]

fig = px.line(
    df_de_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from DE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### From CZ  

In [ ]:
# df_cz_out
df_cz_out = df[(df['country_from'] == 'CZ') & (df['num_migrants'] != 0)]

fig = px.line(
    df_cz_out,
    x="migration_month",
    y="num_migrants",
    color="country_to",  # separate line for each origin country
    markers=True,
    title="Migrants from CZ Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## VE 

In [ ]:
df_ve_in = df[(df['country_to'] == 'VE') & (df['num_migrants'] != 0)]
df_ve_out = df[(df['country_from'] == 'VE') & (df['num_migrants'] != 0)]

In [ ]:
# df_ve_in
fig = px.line(
    df_ve_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to VE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_ve_out
fig = px.line(
    df_ve_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from VE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

#### Plot

##### Waffle for R

In [ ]:
out_migration_totals = df_ve_out.groupby('country_to')['num_migrants'].sum().sort_values(ascending=False)
out_migration_totals

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(out_migration_totals, bins=30, edgecolor='black')

plt.title('Distribution of Migrant Counts per Destination Country')
plt.xlabel('Number of Migrants')
plt.ylabel('Number of Countries')
plt.tight_layout()
plt.show()

In [ ]:
threshold = 0.05 * 1_000_000  # 200,000 migrants
major_countries = out_migration_totals[out_migration_totals > threshold]
others_sum = out_migration_totals[out_migration_totals < threshold].sum()
major_countries['Other'] = others_sum

major_countries

In [ ]:
df_ve_out_top =  df_ve_out[df_ve_out['migration_month'] == '2022-02-01'].sort_values(by='num_migrants', ascending=False).head(11)
df_ve_out_top[['country_to', 'num_migrants']]

In [ ]:
df_month = df_ve_out[df_ve_out['migration_month'] == '2022-02-01']

migration_totals = df_month.groupby('country_to')['num_migrants'].sum()
sorted_totals = migration_totals.sort_values(ascending=False)

top_10 = sorted_totals.head(8)
others_sum = sorted_totals.iloc[8:].sum()

# Use pd.concat instead of append
migration_final = pd.concat([top_10, pd.Series({'Other': others_sum})])

migration_final

##### Heatmap

In [ ]:
# 1. Filter origin and destination countries of interest:
origin_countries = ['VE', 'CL', 'AR', 'PE']
# For destination, you can pick top N countries by migrant volume or specific countries
destination_countries = ['US', 'ES', 'CO', 'BR', 'EC', 'PE', 'CL', 'AR']  # example subset

df_filtered = df[
    (df['country_from'].isin(origin_countries)) &
    (df['country_to'].isin(destination_countries))
]

# 2. Aggregate data: average monthly migrants by origin-destination pair over all months
avg_flows = (
    df_filtered
    .groupby(['country_from', 'country_to'])['num_migrants']
    .mean()
    .reset_index()
)

# 3. Pivot data to get matrix form for heatmap: rows=origin, columns=destination
heatmap_data = avg_flows.pivot(index='country_from', columns='country_to', values='num_migrants')

# 4. Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".0f", cmap='YlGnBu')

plt.title('Average Monthly Migration Flows by Corridor')
plt.ylabel('Origin Country')
plt.xlabel('Destination Country')
plt.tight_layout()
plt.show()


### CO

In [ ]:
df_co_in = df[(df['country_to'] == 'CO') & (df['num_migrants'] != 0)]
df_co_out = df[(df['country_from'] == 'CO') & (df['num_migrants'] != 0)]

In [ ]:
# df_co_in
fig = px.line(
    df_co_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to CO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_co_out
fig = px.line(
    df_co_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from CO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### CL

In [ ]:
df_cl_in = df[(df['country_to'] == 'CL') & (df['num_migrants'] != 0)]
df_cl_out = df[(df['country_from'] == 'CL') & (df['num_migrants'] != 0)]

In [ ]:
# df_cl_in
fig = px.line(
    df_cl_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to CL Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_cl_out
fig = px.line(
    df_cl_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from CL Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### AR

In [ ]:
df_AR_in = df[(df['country_to'] == 'AR') & (df['num_migrants'] != 0)]
df_AR_out = df[(df['country_from'] == 'AR') & (df['num_migrants'] != 0)]

In [ ]:
# df_cl_in
fig = px.line(
    df_AR_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to AR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_cl_out
fig = px.line(
    df_AR_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from AR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### MX

In [ ]:
df_MX_in = df[(df['country_to'] == 'MX') & (df['num_migrants'] != 0)]
df_MX_out = df[(df['country_from'] == 'MX') & (df['num_migrants'] != 0)]

In [ ]:
# df_mx_out
fig = px.line(
    df_MX_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from MX Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_mx_in
fig = px.line(
    df_MX_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to MX Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

### UY

In [ ]:
df_UY_in = df[(df['country_to'] == 'UY') & (df['num_migrants'] != 0)]
df_UY_out = df[(df['country_from'] == 'UY') & (df['num_migrants'] != 0)]

In [ ]:
# df_ur_out
fig = px.line(
    df_UY_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from UY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_uy_in
fig = px.line(
    df_UY_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to UY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## Haiti

In [ ]:
df_HT_in = df[(df['country_to'] == 'HT') & (df['num_migrants'] != 0)]
df_HT_out = df[(df['country_from'] == 'HT') & (df['num_migrants'] != 0)]

In [ ]:
# df_HT_in
fig = px.line(
    df_HT_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to HT Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_HT_out
fig = px.line(
    df_HT_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from HT Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_DO_in = df[(df['country_to'] == 'DO') & (df['num_migrants'] != 0)]
df_DO_out = df[(df['country_from'] == 'DO') & (df['num_migrants'] != 0)]

In [ ]:
# df_DO_in
fig = px.line(
    df_DO_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to DO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_DO_out
fig = px.line(
    df_DO_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from DO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## MM to TH

In [ ]:
df_MM_in = df[(df['country_to'] == 'MM') & (df['num_migrants'] != 0)]
df_MM_out = df[(df['country_from'] == 'MM') & (df['num_migrants'] != 0)]

# df_DO_in
fig = px.line(
    df_MM_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to MM Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_MM_out
fig = px.line(
    df_MM_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from MM Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_SY_in = df[(df['country_to'] == 'SY') & (df['num_migrants'] != 0)]
df_SY_out = df[(df['country_from'] == 'SY') & (df['num_migrants'] != 0)]

# df_DO_in
fig = px.line(
    df_SY_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to SY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_SY_out
fig = px.line(
    df_SY_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from SY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_TR_in = df[(df['country_to'] == 'TR') & (df['num_migrants'] != 0)]
df_TR_out = df[(df['country_from'] == 'TR') & (df['num_migrants'] != 0)]

# df_TR_in
fig = px.line(
    df_TR_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to TR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_TR_out
fig = px.line(
    df_TR_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from TR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

# FB: Prophet

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

## Example 

In [ ]:
# Python
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df.head()
# 2905 rows

In [ ]:
m = Prophet()
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()
# 3270 row

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)

## Actual - CO / VE

In [ ]:
df_ve_co = df_ve_out[df_ve_out['country_to'] == 'CO']
df_co_ve = df_co_out[df_co_out['country_to'] == 'VE']

In [ ]:
# df_ve_co
df_prophet = df_ve_co.copy().rename(columns={
    'migration_month': 'ds',
    'num_migrants': 'y'
})

# Step 2: Convert date to datetime
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Step 3: Initialize Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

# Step 4: Fit the model to your data
model.fit(df_prophet)

# Step 5: Use the same data as the "future" frame (no need to extend it)
forecast = model.predict(df_prophet)

# Step 6: Plot the components (trend and seasonality)
model.plot_components(forecast)
plt.show()

In [ ]:
# df_co_ve
df_prophet = df_co_ve.copy().rename(columns={
    'migration_month': 'ds',
    'num_migrants': 'y'
})

# Step 2: Convert date to datetime
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Step 3: Initialize Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

# Step 4: Fit the model to your data
model.fit(df_prophet)

# Step 5: Use the same data as the "future" frame (no need to extend it)
forecast = model.predict(df_prophet)

# Step 6: Plot the components (trend and seasonality)
model.plot_components(forecast)
plt.show()

## Actual - UA / PL

In [ ]:
df_ua_pl = df[(df['country_to'] == 'PL') & (df['country_from'] == 'UA')]
df_pl_ua = df[(df['country_to'] == 'UA') & (df['country_from'] == 'PL')]

In [ ]:
# df_co_ve
df_prophet = df_ua_pl.copy().rename(columns={
    'migration_month': 'ds',
    'num_migrants': 'y'
})

# Step 2: Convert date to datetime
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Step 3: Initialize Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

# Step 4: Fit the model to your data
model.fit(df_prophet)

# Step 5: Use the same data as the "future" frame (no need to extend it)
forecast = model.predict(df_prophet)

# Step 6: Plot the components (trend and seasonality)
model.plot_components(forecast)
plt.show()

In [ ]:
# df_co_ve
df_prophet = df_pl_ua.copy().rename(columns={
    'migration_month': 'ds',
    'num_migrants': 'y'
})

# Step 2: Convert date to datetime
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Step 3: Initialize Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

# Step 4: Fit the model to your data
model.fit(df_prophet)

# Step 5: Use the same data as the "future" frame (no need to extend it)
forecast = model.predict(df_prophet)

# Step 6: Plot the components (trend and seasonality)
model.plot_components(forecast)
plt.show()

In [ ]:
df_uk_fr = df[(df['country_to'] == 'FR') & (df['country_from'] == 'UK')]
df_fr_uk = df[(df['country_to'] == 'UK') & (df['country_from'] == 'FR')]

In [ ]:
df_prophet = df_uk_fr.copy().rename(columns={
    'migration_month': 'ds',
    'num_migrants': 'y'
})

# Step 2: Convert date to datetime
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Step 3: Initialize Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

# Step 4: Fit the model to your data
model.fit(df_prophet)

# Step 5: Use the same data as the "future" frame (no need to extend it)
forecast = model.predict(df_prophet)

# Step 6: Plot the components (trend and seasonality)
model.plot_components(forecast)
plt.show()

In [ ]:
df_prophet = df_fr_uk.copy().rename(columns={
    'migration_month': 'ds',
    'num_migrants': 'y'
})

# Step 2: Convert date to datetime
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# Step 3: Initialize Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)

# Step 4: Fit the model to your data
model.fit(df_prophet)

# Step 5: Use the same data as the "future" frame (no need to extend it)
forecast = model.predict(df_prophet)

# Step 6: Plot the components (trend and seasonality)
model.plot_components(forecast)
plt.show()

# Test: with Baysian

In [ ]:
# Step 1: Create unique flow ID (origin -> destination)
df["flow_id"] = df["country_from"] + "_" + df["country_to"]

# Create numeric indices for flows and months
df["flow_index"] = df["flow_id"].astype("category").cat.codes
df["time_index"] = (
    (df["migration_month"].dt.year - df["migration_month"].dt.year.min()) * 12 +
    (df["migration_month"].dt.month - df["migration_month"].dt.month.min())
)

# Check unique flow mappings and time indexing
flow_map = df[["flow_id", "flow_index"]].drop_duplicates().reset_index(drop=True)
time_range = df["migration_month"].min(), df["migration_month"].max()
df

In [ ]:
# Your data:
flow_idx = df["flow_index"].values
time_idx = df["time_index"].values
y = df["num_migrants"].values
num_flows = df["flow_index"].nunique()
num_times = df["time_index"].nunique()

with pm.Model() as model:
    mu = pm.Normal("mu", mu=0, sigma=5)
    sigma_flow = pm.HalfNormal("sigma_flow", sigma=2)
    alpha = pm.Normal("alpha", mu=0, sigma=sigma_flow, shape=num_flows)
    sigma_time = pm.HalfNormal("sigma_time", sigma=2)
    beta = pm.Normal("beta", mu=0, sigma=sigma_time, shape=num_times)
    
    log_lambda = mu + alpha[flow_idx] + beta[time_idx]
    lambda_ = pm.math.exp(log_lambda)
    
    y_obs = pm.Poisson("y_obs", mu=lambda_, observed=y)

    trace = pm.sample(2000, tune=2000, target_accept=0.99, 
                      init='jitter+adapt_diag', chains=4, cores=4, max_treedepth=15, return_inferencedata=True)
    
    # trace = pm.sample(500, tune=500, chains=2, cores=2, target_accept=0.9)  # for simple test


| Term            | Meaning                                                                    |
| --------------- | -------------------------------------------------------------------------- |
| **mean**        | Posterior mean (best guess based on all chains)                            |
| **sd**          | Posterior standard deviation (uncertainty)                                 |
| **hdi\_3%-97%** | 94% credible interval (Bayesian version of confidence interval)            |
| **r\_hat**      | Should be \~1.00. Values >1.1 → **non-converged** (⚠️ yours are too high)  |
| **ess\_bulk**   | Effective sample size. You want this to be **much higher** (like >200)     |
| **trace plots** | Chain mixing over time — your `mu` and `sigma_*` are **not well-mixed** 😕 |


In [ ]:
az.plot_trace(trace, var_names=["mu", "sigma_flow", "sigma_time"])
az.summary(trace, var_names=["mu", "sigma_flow", "sigma_time"])

In [ ]:
# import numpy as np
# import arviz as az

# Extract posterior samples from trace
mu_samples = trace.posterior["mu"].values.flatten()
alpha_samples = trace.posterior["alpha"].values[:, :, 28].flatten()  # flow_index = 36 UK; 28 PL
beta_samples = trace.posterior["beta"].values[:, :, 38].flatten()    # time_index = 41

# Compute log-lambda samples
log_lambda_samples = mu_samples + alpha_samples + beta_samples

# Convert to rate (expected migrant count)
lambda_samples = np.exp(log_lambda_samples)

# Summary statistics
mean_prediction = lambda_samples.mean()
median_prediction = np.median(lambda_samples)
hdi_interval = az.hdi(lambda_samples, hdi_prob=0.94)

print(f"Estimated migrants from UA to UK in 2022-04:")
print(f"  Posterior mean: {mean_prediction:.2f}")
print(f"  Posterior median: {median_prediction:.2f}")
print(f"  94% credible interval: {hdi_interval[0]:.2f} to {hdi_interval[1]:.2f}")


In [ ]:
# df[(df['country_from'] == 'UA') & (df['country_to'] == 'PL')]
# df[(df['country_from'] == 'UA') & (df['country_to'] == 'UK')]
df[(df['flow_index'] == 28) & (df['time_index'] == 38)]